# Install necessary packages

In [208]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-24.3.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Program Files\Python312\python.exe -m pip install --upgrade pip


In [242]:
!pip install -U torch numpy==1.26 pandas scikit-learn plotly nltk transformers==4.46.3 sentence-transformers einops datasets gradio networkx umap-learn ipywidgets

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.45.0-py3-none-any.whl.metadata (44 kB)
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   ------------------ --------------------- 1.0/2.2 MB 3.0 MB/s eta 0:00:01

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.15 which is incompatible.
langchain-huggingface 0.0.3 requires tokenizers>=0.19.1, but you have tokenizers 0.19.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Libraries

In [210]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from sklearn.manifold import TSNE
from IPython.display import display
import umap

# Download NLTK resources

In [211]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load the CSV files to inspect their contents

In [212]:
current_dir = os.path.join(os.getcwd(), 'data')

In [213]:
program_df = pd.read_csv(os.path.join(current_dir, 'Program.csv'))
program_df['title'] = program_df['title'].str.replace(r'<[^>]*>', '', regex=True)
program_course_df = pd.read_csv(os.path.join(current_dir, 'ProgramCourse.csv'))
program_type_df = pd.read_csv(os.path.join(current_dir, 'ProgramType.csv'))
course_df = pd.read_csv(os.path.join(current_dir, 'Course.csv'))

# Inspect the DataFrames

In [214]:
print("Program DataFrame Head:")
program_df['title'] = program_df['title'].str.replace(r'<[^>]*>', '', regex=True)
display(program_df.head(200))


Program DataFrame Head:


,code,credits,horaireCoursPdfJson,planificationPdfJson,createdAt,updatedAt,title,url,cycle,id
0,0648,15 crédits,NaN,NaN,2024-10-17 04:32:25.190,2024-11-02 01:50:20.409,Programme court de 2e cycle en génie de l'envi...,https://www.etsmtl.ca/programmes-formations/pr...,2,183146
1,0569,15 crédits,NaN,NaN,2024-10-17 04:32:25.191,2024-11-02 01:50:20.409,Programme court de 2e cycle en génie de la pro...,https://www.etsmtl.ca/programmes-formations/pr...,2,183156
2,0514,15 crédits,NaN,NaN,2024-10-17 04:32:25.191,2024-11-02 01:50:20.409,Programme court de 2e cycle en gestion de l’in...,https://www.etsmtl.ca/programmes-formations/pr...,2,183236
3,3294,30 crédits,NaN,NaN,2024-10-17 04:32:25.184,2024-11-02 01:50:20.408,DESS en projets internationaux et ingénierie g...,https://www.etsmtl.ca/programmes-formations/de...,2,183498
4,6646,"90 crédits, incluant 9 crédits de stage",NaN,NaN,2024-10-17 04:32:25.184,2024-11-02 01:50:20.408,Baccalauréat en informatique distribuée,https://www.etsmtl.ca/programmes-formations/ba...,1,182928
...,...,...,...,...,...,...,...,...,...,...
89,1682,30 crédits,NaN,NaN,2024-10-17 04:32:25.184,2024-11-02 01:50:20.408,DESS en génie des risques de santé et sécurité...,https://www.etsmtl.ca/programmes-formations/de...,2,183424
90,3209,30 crédits,NaN,NaN,2024-10-17 04:32:25.184,2024-11-02 01:50:20.408,DESS en génie de la production automatisée : i...,https://www.etsmtl.ca/programmes-formations/de...,2,183404
91,0854,15 crédits,NaN,NaN,2024-10-17 04:32:25.191,2024-11-02 01:50:20.409,Programme court de 2e cycle en génie de la con...,https://www.etsmtl.ca/programmes-formations/pr...,2,183136
92,9081,15 crédits,NaN,NaN,2024-10-17 04:32:25.192,2024-11-02 01:50:20.409,Programme court de 2e cycle en ingénierie fina...,https://www.etsmtl.ca/programmes-formations/pr...,2,183276


In [215]:
print("\nProgramCourse DataFrame Head:")
display(program_course_df.head())


ProgramCourse DataFrame Head:


,createdAt,updatedAt,typicalSessionIndex,courseId,programId,type
0,2024-11-02 01:50:21.887,2024-11-02 01:50:21.887,1,351029,183256,NaN
1,2024-11-02 01:50:21.913,2024-11-02 01:50:21.913,3,353404,183040,TRONC
2,2024-11-02 01:50:21.958,2024-11-02 01:50:21.958,4,352245,182976,TRONC
3,2024-11-02 01:50:21.973,2024-11-02 01:50:21.973,4,351827,182976,TRONC
4,2024-11-02 01:50:21.981,2024-11-02 01:50:21.981,5,353458,182976,TRONC


In [216]:
print("\nProgramType DataFrame Head:")
display(program_type_df.head())


ProgramType DataFrame Head:


,id,title
0,697435,Maîtrise avec projet
1,738239,Microprogramme
2,697451,Maîtrise avec mémoire
3,915770,Concentration en technologies de la santé
4,697388,Doctorat


In [217]:
print("\nCourse DataFrame Head:")
display(course_df.head())


Course DataFrame Head:


,code,title,description,credits,createdAt,updatedAt,id,cycle
0,ATE800E,Academic Integrity : Concepts and Techniques,The ATE800 workshop must be passed in the firs...,0,2024-11-02 01:50:21.499,2024-11-02 01:50:21.569,407641,2
1,ELE735,Analyse numérique,"Au terme de ce cours, l'étudiante ou l'étudian...",3,2024-11-02 01:50:21.502,2024-11-02 01:50:21.773,350543,1
2,MTI850,Analytiques des données massives,Ce cours présente les concepts pour effectuer ...,3,2024-11-02 01:50:21.502,2024-11-02 01:50:21.773,353344,2
3,CHM015,Chimie préparatoire pour le génie (hors progra...,Ce cours vise à initier l’étudiante ou l'étudi...,3,2024-11-02 01:50:21.503,2024-11-02 01:50:21.773,349708,1
4,ELE752,Appareillage électrique,"Au terme de ce cours, l'étudiante ou l'étudian...",3,2024-11-02 01:50:21.502,2024-11-02 01:50:21.773,350599,1


In [218]:
print("\nDataFrame Columns:")
print("Program:", program_df.columns.tolist())
print("ProgramCourse:", program_course_df.columns.tolist())
print("ProgramType:", program_type_df.columns.tolist())
print("Course:", course_df.columns.tolist())


DataFrame Columns:
Program: ['code', 'credits', 'horaireCoursPdfJson', 'planificationPdfJson', 'createdAt', 'updatedAt', 'title', 'url', 'cycle', 'id']
ProgramCourse: ['createdAt', 'updatedAt', 'typicalSessionIndex', 'courseId', 'programId', 'type']
ProgramType: ['id', 'title']
Course: ['code', 'title', 'description', 'credits', 'createdAt', 'updatedAt', 'id', 'cycle']


# How many courses are in the dataset?

In [219]:
num_courses = course_df.shape[0]
print(f"Number of courses in the dataset: {num_courses}")

Number of courses in the dataset: 907


# How many program are in the dataset?

In [220]:
num_programs = program_df.shape[0]
print(f"Number of programs in the dataset: {num_programs}")

Number of programs in the dataset: 94


In [221]:
num_program_courses = program_course_df.shape[0]
print(f"Number of program courses in the dataset: {num_program_courses}")

Number of program courses in the dataset: 692


In [222]:
num_program_courses = program_course_df.shape[0]
print(f"Number of program courses in the dataset: {num_program_courses}")

Number of program courses in the dataset: 692


In [223]:
num_program_types = program_type_df.shape[0]
print(f"Number of program types in the dataset: {num_program_types}")

Number of program types in the dataset: 10


# Combine the Program and ProgramCourse DataFrames

In [224]:
# Ensure consistent data types for merge keys
program_course_df['courseId'] = program_course_df['courseId'].astype(str)
course_df['id'] = course_df['id'].astype(str)

program_course_df['programId'] = program_course_df['programId'].astype(str)
program_df['id'] = program_df['id'].astype(str)

# Merge DataFrames with suffixes to differentiate columns

In [225]:
combined_df = pd.merge(
    program_course_df,
    program_df,
    left_on='programId',
    right_on='id',
    how='left',
    suffixes=('_program_course', '_program')
)

# Print columns after first merge
print("Columns after merging program_course_df and program_df:")
print(combined_df.columns.tolist())


Columns after merging program_course_df and program_df:
['createdAt_program_course', 'updatedAt_program_course', 'typicalSessionIndex', 'courseId', 'programId', 'type', 'code', 'credits', 'horaireCoursPdfJson', 'planificationPdfJson', 'createdAt_program', 'updatedAt_program', 'title', 'url', 'cycle', 'id']


# Merge with course_df

In [226]:
combined_df = pd.merge(
    combined_df,
    course_df,
    left_on='courseId',
    right_on='id',
    how='left',
    suffixes=('', '_course')
)


# Print columns after second merge

In [227]:
print("Columns after merging with course_df:")
print(combined_df.columns.tolist())

Columns after merging with course_df:
['createdAt_program_course', 'updatedAt_program_course', 'typicalSessionIndex', 'courseId', 'programId', 'type', 'code', 'credits', 'horaireCoursPdfJson', 'planificationPdfJson', 'createdAt_program', 'updatedAt_program', 'title', 'url', 'cycle', 'id', 'code_course', 'title_course', 'description', 'credits_course', 'createdAt', 'updatedAt', 'id_course', 'cycle_course']


# Optional: Rename columns for clarity

In [228]:
combined_df.rename(columns={
    'title': 'title_program',
    'code': 'code_program',
    'cycle': 'cycle_program',
    'credits': 'credits_program',
    'horaireCoursPdfJson': 'horaireCoursPdfJson_program',
    'title_course': 'title_course',
    'code_course': 'code_course',
    'cycle_course': 'cycle_course',
    'credits_course': 'credits_course',
    'description': 'description_course'
}, inplace=True)


# Updated list of columns to include in the combined text

In [229]:
columns_to_include = [
    'programId', 'courseId', 'type',
    'title_program', 'code_program', 'cycle_program', 'credits_program', 'horaireCoursPdfJson_program',
    'title_course', 'code_course', 'cycle_course', 'credits_course', 'description_course'
]


# Drop createdAt_program_course updatedAt_program_course horaireCoursPdfJson_program planificationPdfJson

In [230]:

columns_to_drop = [
    'createdAt_program_course', 
    'updatedAt_program_course', 
    'horaireCoursPdfJson', 
    'planificationPdfJson'
]

combined_df = combined_df.drop(columns=columns_to_drop, errors='ignore')

# Find all the courses for each program

In [231]:
courses_per_program = combined_df.groupby('programId').agg({
    'title_course': list,  # Course titles
    'title_program': 'first'  # Program title
}).reset_index()

In [232]:
courses_per_program.rename(columns={'title': 'course_titles', 'title_program': 'program_title'}, inplace=True)
courses_per_program

,programId,title_course,program_title
0,182816,[Éléments de gestion de projets de constructio...,Baccalauréat en génie de la construction
1,182848,"[Calcul différentiel et intégral, Statique et ...",Baccalauréat en génie logiciel
2,182864,"[Règles de base en santé et sécurité, Développ...",Baccalauréat en génie mécanique
3,182880,[Comportement organisationnel et performance d...,Baccalauréat en génie des opérations et de la ...
4,182896,"[Calcul différentiel et intégral, Règles de ba...",Baccalauréat en génie de la production automat...
5,182912,"[Calcul différentiel et intégral, Développemen...",Baccalauréat en génie des technologies de l'in...
6,182952,"[Intégrité intellectuelle, Règles de base en s...",Certificat en économie et estimation des proje...
7,182960,"[Règles de base en santé et sécurité, Intégrit...",Certificat en gestion et assurance de la qualité
8,182976,"[Gestion des actifs immobiliers, Gestion du pe...",Certificat en gestion immobilière
9,182984,"[Intégrité intellectuelle, Informatique et sta...",Certificat en production industrielle


# How course per program for each program

In [233]:
# for each programId, count the number of courses title_coursein the list
courses_per_program['num_courses'] = courses_per_program['title_course'].apply(len)
courses_per_program

,programId,title_course,program_title,num_courses
0,182816,[Éléments de gestion de projets de constructio...,Baccalauréat en génie de la construction,64
1,182848,"[Calcul différentiel et intégral, Statique et ...",Baccalauréat en génie logiciel,59
2,182864,"[Règles de base en santé et sécurité, Développ...",Baccalauréat en génie mécanique,81
3,182880,[Comportement organisationnel et performance d...,Baccalauréat en génie des opérations et de la ...,51
4,182896,"[Calcul différentiel et intégral, Règles de ba...",Baccalauréat en génie de la production automat...,67
5,182912,"[Calcul différentiel et intégral, Développemen...",Baccalauréat en génie des technologies de l'in...,62
6,182952,"[Intégrité intellectuelle, Règles de base en s...",Certificat en économie et estimation des proje...,13
7,182960,"[Règles de base en santé et sécurité, Intégrit...",Certificat en gestion et assurance de la qualité,17
8,182976,"[Gestion des actifs immobiliers, Gestion du pe...",Certificat en gestion immobilière,3
9,182984,"[Intégrité intellectuelle, Informatique et sta...",Certificat en production industrielle,15


In [234]:
dess_programs = courses_per_program[courses_per_program['program_title'].str.contains("DESS", case=False, na=False)]

In [235]:
dess_programs

,programId,title_course,program_title,num_courses
30,183344,[Modélisation énergétique (6D) et gestion (7D)...,DESS en BIM et innovations numériques,7
31,183354,"[Commande par micro-ordinateur, Systèmes de me...",DESS en énergies renouvelables et efficacité é...,5
32,183364,[Fabrication additive de composants à forte va...,DESS en entreprise numérique,2
33,183384,[Intégrité intellectuelle : un savoir-être et ...,DESS en génie de la construction : gestion de ...,16
34,183394,[Fondements des procédés et processus en génie...,DESS en génie de l'environnement,3
35,183404,[Gestion de projets dans les produits et servi...,DESS en génie de la production automatisée : i...,10
36,183414,[Gestion de projets dans les produits et servi...,DESS en génie de la production automatisée : s...,10
37,183424,"[Ergonomie et sécurité en milieu industriel, É...",DESS en génie des risques de santé et sécurité...,4
38,183464,[Introduction à l’entrepreneurship],DESS en gestion de l’innovation,1
39,183508,"[Signaux et systèmes numériques, Processeur nu...",DESS en réseaux de télécommunications,3


In [236]:
bachelor_programs = courses_per_program[courses_per_program['program_title'].str.contains("Baccalauréat", case=False, na=False)]

In [237]:
bachelor_programs

,programId,title_course,program_title,num_courses
0,182816,[Éléments de gestion de projets de constructio...,Baccalauréat en génie de la construction,64
1,182848,"[Calcul différentiel et intégral, Statique et ...",Baccalauréat en génie logiciel,59
2,182864,"[Règles de base en santé et sécurité, Développ...",Baccalauréat en génie mécanique,81
3,182880,[Comportement organisationnel et performance d...,Baccalauréat en génie des opérations et de la ...,51
4,182896,"[Calcul différentiel et intégral, Règles de ba...",Baccalauréat en génie de la production automat...,67
5,182912,"[Calcul différentiel et intégral, Développemen...",Baccalauréat en génie des technologies de l'in...,62


In [238]:
master_programs = courses_per_program[courses_per_program['program_title'].str.contains("Maîtrise", case=False, na=False)]
master_programs

,programId,title_course,program_title,num_courses


In [239]:
doctorate_programs = courses_per_program[courses_per_program['program_title'].str.contains("Doctorat", case=False, na=False)]

In [240]:
doctorate_programs

,programId,title_course,program_title,num_courses
41,183738,"[Sujets spéciaux II en génie mécanique, Sujets...",Doctorat en génie,2
